# DeepGTA - Multi-Object Tracking Demo

This notebook demonstrates the DeepGTA pipeline for multi-object tracking.

**Pipeline:**
- YOLOv11x Detection with configurable class handling
- Deep-EIoU Online Tracking with ReID features
- GTA-Link Offline Refinement

[![GitHub](https://img.shields.io/badge/GitHub-DeepGTA-blue)](https://github.com/HiteshG/DeepGTA)

## 1. Installation

In [ ]:
# Clone DeepGTA repository
!git clone https://github.com/HiteshG/DeepGTA.git
%cd DeepGTA

# Install dependencies
!pip install -q -r requirements.txt
!pip install -q lap cython-bbox gdown

print("Installation complete!")

## 2. Configuration

Configure the pipeline for your use case. Below is an example for hockey tracking.

In [ ]:
from deepgta import DeepGTAConfig, DeepGTAPipeline

# Example: Hockey tracking configuration
config = DeepGTAConfig(
    # === Detection Settings ===
    yolo_weights="/content/HockeyAI_model_weight.pt",  # Upload your model
    class_names=[
        "Center Ice", "Faceoff", "Goalpost",
        "Goaltender", "Player", "Puck", "Referee"
    ],
    track_classes=[3, 4, 5, 6],   # Goaltender, Player, Puck, Referee
    special_classes=[5],           # Puck - max-conf only
    detection_conf_thresh=0.5,
    detection_iou_thresh=0.7,

    # === Tracking Settings ===
    with_reid=True,
    reid_model="/content/sports_model.pth.tar-60",  # Optional
    track_high_thresh=0.7,
    track_low_thresh=0.4,
    track_buffer=90,

    # === Refinement Settings ===
    use_refinement=True,
    split_eps=0.65,
    split_min_samples=15,
    merge_dist_thresh=0.35,

    # === Output Settings ===
    output_video=True,
    draw_tracks=True,
    draw_labels=True,
    verbose=True,
)

print("Configuration created!")
print(f"Tracking classes: {[config.class_names[i] for i in config.track_classes]}")
print(f"Special classes: {[config.class_names[i] for i in config.special_classes]}")

## 3. Upload Files

Upload your video and model weights.

In [ ]:
from google.colab import files

print("Upload your files:")
print("1. YOLO model weights (.pt)")
print("2. Input video (.mp4)")
print("3. (Optional) ReID model weights")

uploaded = files.upload()

for filename in uploaded.keys():
    print(f"Uploaded: {filename}")

### (Optional) Download Default ReID Model

In [ ]:
# Download default sports ReID model
import gdown

reid_url = "https://drive.google.com/uc?id=14zzlm1nI9Ws_Il9RYNChwPC7Fsul7xwl"
reid_output = "/content/sports_model.pth.tar-60"

gdown.download(reid_url, reid_output, quiet=False)
print(f"ReID model downloaded to: {reid_output}")

## 4. Run Pipeline

In [ ]:
# Update config with actual file paths
config.yolo_weights = "/content/HockeyAI_model_weight.pt"  # Update this
config.reid_model = "/content/sports_model.pth.tar-60"     # Update this

# Create pipeline
pipeline = DeepGTAPipeline(config)

# Process video
input_video = "/content/input_video.mp4"  # Update this
output_video = "/content/output_tracked.mp4"

output_path, tracklets = pipeline.process_video(
    video_path=input_video,
    output_path=output_video,
    show_progress=True
)

print(f"\nTracking complete!")
print(f"Output saved to: {output_path}")
print(f"Total unique tracks: {len(tracklets)}")

## 5. Display Results

In [ ]:
import subprocess
from IPython.display import HTML
from base64 import b64encode

# Compress video for display
compressed_video = "/content/output_compressed.mp4"
subprocess.run([
    "ffmpeg", "-i", output_video,
    "-vcodec", "libx264", "-crf", "28",
    "-preset", "fast",
    compressed_video, "-y"
], capture_output=True)

# Display in notebook
mp4 = open(compressed_video, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'''
<video width="800" controls>
    <source src="{data_url}" type="video/mp4">
</video>
''')

## 6. Analyze Results

In [ ]:
# Print tracking statistics
print("=" * 50)
print("Tracking Statistics")
print("=" * 50)

# Count tracks by class
class_counts = {}
total_detections = 0

for tid, tracklet in tracklets.items():
    class_id = tracklet.class_id
    class_name = config.class_names[class_id] if class_id < len(config.class_names) else f"class_{class_id}"

    if class_name not in class_counts:
        class_counts[class_name] = {'tracks': 0, 'detections': 0}

    class_counts[class_name]['tracks'] += 1
    class_counts[class_name]['detections'] += len(tracklet.times)
    total_detections += len(tracklet.times)

print(f"\nTotal unique tracks: {len(tracklets)}")
print(f"Total detections: {total_detections}")
print(f"\nBy class:")
for class_name, counts in sorted(class_counts.items()):
    print(f"  {class_name}: {counts['tracks']} tracks, {counts['detections']} detections")

# Track lengths
track_lengths = [len(t.times) for t in tracklets.values()]
print(f"\nTrack length statistics:")
print(f"  Min: {min(track_lengths)} frames")
print(f"  Max: {max(track_lengths)} frames")
print(f"  Mean: {sum(track_lengths) / len(track_lengths):.1f} frames")

## 7. Download Results

In [ ]:
from google.colab import files

# Download tracked video
files.download(output_video)

# Optionally download MOT format results
# if config.save_mot_txt:
#     files.download(config.mot_output_path)

---

## Custom Configuration Examples

### Soccer Tracking

In [ ]:
# Soccer tracking configuration
soccer_config = DeepGTAConfig(
    yolo_weights="/content/soccer_model.pt",
    class_names=["Background", "Player", "Ball", "Referee", "Goalkeeper"],
    track_classes=[1, 2, 3, 4],
    special_classes=[2],  # Ball - single instance
    detection_conf_thresh=0.4,

    # Longer buffer for soccer (players may be occluded longer)
    track_buffer=120,

    with_reid=True,
    use_refinement=True,
)

### Without Refinement (Faster Processing)

In [ ]:
# Fast mode - no refinement
fast_config = DeepGTAConfig(
    yolo_weights="/content/model.pt",
    class_names=["Person", "Car", "Truck"],
    track_classes=[0, 1, 2],

    # Disable refinement for faster processing
    use_refinement=False,

    # Disable ReID for even faster processing
    with_reid=False,

    output_video=True,
)